# Selwyn Well I36_0477 Worked example

Before we get too far, lets import the necessary libraries and set up the the notebook

## Background

## Step 1 assess the historical data

### Look and remove any outliers 

### Identify any historical trend

## Step 2: estimate the noise level

## Step 3: Estimate the likely historical source concentration

## Step 4: Generate the future scenarios


### Create the true receptor time series




## Step 5: Estimate the detection power of the monitoring network


### Problems with Slope detection power


### Run Change detection power instead